In [ ]:
import os
import numpy as np
from skimage import io, util
import cv2
import pandas as pd
from datetime import datetime

def add_gaussian_noise(image, var=0.01):
    return util.random_noise(image, mode='gaussian', var=var)

def add_salt_and_pepper_noise(image, amount=0.1):
    return util.random_noise(image, mode='s&p', amount=amount)

def add_poisson_noise(image):
    return util.random_noise(image, mode='poisson')

def add_speckle_noise(image, var=0.01):
    return util.random_noise(image, mode='speckle', var=var)

def update_excel_record(main_dir, output_dir):
    # Define the path to the Excel file
    excel_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'processed_datasets.xlsx')
    
    # Create a DataFrame with the new record
    new_record = pd.DataFrame({
        'Main Directory': [main_dir],
        'Output Directory': [output_dir],
        'Timestamp': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
    })
    
    # Check if the Excel file already exists
    if os.path.exists(excel_path):
        # Load the existing Excel file
        existing_records = pd.read_excel(excel_path)
        # Append the new record
        updated_records = pd.concat([existing_records, new_record], ignore_index=True)
    else:
        # If the file does not exist, the new record is the only record
        updated_records = new_record
    
    # Save the updated records to the Excel file
    updated_records.to_excel(excel_path, index=False)

def process_images(main_dir):
    # Define the output directory as a subdirectory of the main_dir's parent directory
    output_dir = os.path.join(os.path.dirname(main_dir), 'complete_dl_data')

    # Define the subdirectories
    subdirs = ['train', 'valid', 'test']

    # Define the noise types and corresponding functions with 10% noise factor
    noise_types = {
        'gaussian': lambda image: add_gaussian_noise(image, var=0.01),
        'salt_and_pepper': lambda image: add_salt_and_pepper_noise(image, amount=0.1),
        'poisson': add_poisson_noise,  # Poisson noise does not take a noise factor
        'speckle': lambda image: add_speckle_noise(image, var=0.01)
    }

    for subdir in subdirs:
        subdir_path = os.path.join(main_dir, subdir)
        output_subdir_path = os.path.join(output_dir, subdir)
        
        # Create new directories for each noise type in the output directory
        for noise_type in noise_types.keys():
            noise_dir = os.path.join(output_subdir_path, noise_type)
            os.makedirs(noise_dir, exist_ok=True)
        
        # Create directory for original images
        original_dir = os.path.join(output_subdir_path, 'original')
        os.makedirs(original_dir, exist_ok=True)
        
        # Process each image in the subdirectory
        for class_dir in os.listdir(subdir_path):
            class_dir_path = os.path.join(subdir_path, class_dir)
            if os.path.isdir(class_dir_path):
                for file_name in os.listdir(class_dir_path):
                    file_path = os.path.join(class_dir_path, file_name)
                    if os.path.isfile(file_path):
                        try:
                            image = io.imread(file_path)
                            image = image.astype(np.float32) / 255.0  # Normalize image to range [0, 1]
                            
                            for noise_type, noise_func in noise_types.items():
                                noisy_image = noise_func(image)
                                noisy_image = np.clip(noisy_image, 0, 1)  # Clip to range [0, 1]
                                noisy_image = (noisy_image * 255).astype(np.uint8)  # Convert to uint8 format
                                noisy_image_bgr = cv2.cvtColor(noisy_image, cv2.COLOR_RGB2BGR)
                                noise_dir = os.path.join(output_subdir_path, noise_type)
                                noisy_image_path = os.path.join(noise_dir, file_name)
                                cv2.imwrite(noisy_image_path, noisy_image_bgr)
                            
                            # Save the original image in the output directory
                            original_image = (image * 255).astype(np.uint8)  # Convert to uint8 format
                            original_image_bgr = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)
                            original_image_path = os.path.join(original_dir, file_name)
                            cv2.imwrite(original_image_path, original_image_bgr)
                        except Exception as e:
                            print(f"Error processing file {file_path}: {e}")

    # Update the Excel record
    update_excel_record(main_dir, output_dir)
    print("Images processed and saved with noise.")

# Example usage
main_dir = r'C:\Users\Qwerty\Downloads\Benchmark_dataset\MNIST\dl_data'
process_images(main_dir)